### Final

In [1]:
# features path
import pandas as pd
rpath = '/'
tr_df = pd.read_csv(rpath + 'tr_python_features')
tr_df = tr_df.merge(pd.read_csv(rpath + 'smoothed_hadoop_features_tr'), how='left')

tt_df = pd.read_csv(rpath + 'tt_python_features')
tt_df = tt_df.merge(pd.read_csv(rpath + 'smoothed_hadoop_features_tt'), how='left')

In [2]:
tr_df_copy = tr_df.copy()
tr_df_copy = tr_df_copy.drop(columns=['DocumentId', 'QueryId', 'mark'])

tt_df_copy = tt_df.copy()
tt_df_copy = tt_df_copy.drop(columns=['DocumentId', 'QueryId'])

In [3]:
!pip install catboost

In [4]:
X_train = tr_df_copy.values
y_train = tr_df["mark"].values
X_test = tt_df_copy.values

In [5]:
import numpy as np

trainqids = set(tr_df.QueryId.tolist())
np.random.seed(42)
train_ind = np.random.choice(list(trainqids), int(0.8 * len(trainqids))).tolist()
test_ind = list(trainqids - set(train_ind))

In [6]:
valid_train = tr_df[tr_df.QueryId.apply(lambda x: x in train_ind)]
valid_test = tr_df[tr_df.QueryId.apply(lambda x: x in test_ind)]

In [7]:
from catboost import CatBoost, Pool, MetricVisualizer
train_pool = Pool(
    data = tr_df_copy[tr_df.QueryId.apply(lambda x: x in train_ind)].values,
    label = valid_train.mark.values / 5,
    group_id = valid_train.QueryId.values
)

valid_pool = Pool(
    data = tr_df_copy[tr_df.QueryId.apply(lambda x: x in test_ind)].values,
    label = valid_test.mark.values / 5,
    group_id = valid_test.QueryId.values
)

full = Pool(
    data = tr_df_copy.values,
    label = tr_df.mark.values / 5,
    group_id = tr_df.QueryId.values
)

In [9]:
parameters = {
    'iterations': 1300,
    'custom_metric': ['NDCG', 'PFound', 'AverageGain:top=5'],
    'verbose': False,
    'random_seed': 543434,
    'depth': 8,
    'bootstrap_type': 'Bernoulli',
    'loss_function': 'YetiRank',
    'subsample': 0.8
}


In [10]:
from copy import deepcopy

model = CatBoost(parameters)
model.fit(full, eval_set = [full,valid_pool], plot=True)

In [11]:
test_data = Pool(data = X_test)
tt_df["mark_cb"] = model.predict(test_data)
result = tt_df.sort_values(by=['QueryId', "mark_cb"], ascending = [True, False])

In [13]:
with open('submission', 'w') as f:
    f.write("QueryId,DocumentId\n")
    for qid in set(result["QueryId"].tolist()):
        cur_res = result[result["QueryId"] == qid]["DocumentId"].tolist()
        c = 0
        started_ids = set()
        for uid in cur_res:
            c += 1
            started_ids.add(uid)
            f.write("{},{}\n".format(qid, uid))
            if c == 5: 
                break
        if c < 5:
            not_finished = set(test[test["qid"] == 0]["DocumentId"].tolist()) - started_ids
            for uid in not_finished:
                c += 1
                f.write("{},{}\n".format(qid, uid))
                if c == 5:
                    break